## Тема 2. “Создание признакового пространства”

#### 1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.

●	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.

●	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.

●	Исключим стоп-слова с помощью stop_words='english'.

●	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().

#### 2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.

●	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.

●	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.

●	Исключим стоп-слова с помощью stop_words='english'.

●	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().

#### 3. Натренируем gensim.models.Word2Vec модель на наших данных.

●	Тренировать будем на токенизированных твитах combine_df['tweet_token']

●	Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.

●	Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.

#### 4. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".

#### 5. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).
Давайте проверим векторное представление любого слова из нашего корпуса, например "food".

#### 6. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.

Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size))
и поделить финальный вектор на количество слов в твите.
На выходе должен получиться wordvec_df.shape = (49159, 200).


In [4]:
import numpy as np
import pandas as pd
import pickle

## Задание 1
Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.

In [9]:
df = pd.read_pickle("../01/tweets.pkl")

In [10]:
df.head(2)

,id,label,tweet,tweet_token,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,"[thanks, for, lyft, credit, can, not, use, cau...","[thank, lyft, credit, use, caus, offer, wheelc...","[thank, lyft, credit, use, cause, offer, wheel..."


In [41]:
from sklearn.feature_extraction.text import CountVectorizer

In [42]:
texts = df['tweet_stemmed']
texts = texts.to_list()
texts[:4]

[['father', 'dysfunct', 'selfish', 'drag', 'kid', 'dysfunct', 'run'],
 ['thank',
  'lyft',
  'credit',
  'use',
  'caus',
  'offer',
  'wheelchair',
  'van',
  'pdx',
  'disapoint',
  'getthank'],
 ['bihday', 'majesti'],
 ['model', 'love', 'take', 'time', 'ur']]

In [43]:
# приводим тексты твитов в формат, который принимает векторайзер (список предложений)
all_text_stemmed = []
for row in texts:
    row_text = ""
    for word in row:
        row_text = row_text + " " + word
    all_text_stemmed.append(row_text.strip())

In [44]:
# что у нас получилось после преобразования
all_text_stemmed[:10]

['father dysfunct selfish drag kid dysfunct run',
 'thank lyft credit use caus offer wheelchair van pdx disapoint getthank',
 'bihday majesti',
 'model love take time ur',
 'factsguid societi motiv',
 'huge fan fare big talk leav chao pay disput get allshowandnogo',
 'camp tomorrow danni',
 'next school year year exam think school exam hate imagin actorslif revolutionschool girl',
 'love land allin cav champion cleveland clevelandcavali',
 'welcom gr']

In [45]:
# создаём объект CountVectorizer и устанавливаем его параметры
# Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
# Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
# Исключим стоп-слова с помощью stop_words='english'.
count_vectorizer = CountVectorizer(ngram_range=(1, 1), analyzer='word', 
                                   binary=False, max_df = 0.9, max_features = 1000,
                                  stop_words='english')

In [46]:
# Создаем the Bag-of-Words модель
bag_of_words = count_vectorizer.fit_transform(all_text_stemmed)

In [47]:
# Отобразим Bag-of-Words модель как DataFrame
feature_names = count_vectorizer.get_feature_names()
bow_df = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)

In [48]:
# проверяем наш датафрейм, созданный на BoW
bow_df.head(2)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Проделаем то же самое для лемм из столбца "tweet_lemmatized"

In [49]:
# достаём все тексты из поля tweet_lemmatized
texts = df['tweet_lemmatized']
texts = texts.to_list()
texts[:4]

[['father', 'dysfunctional', 'selfish', 'drag', 'kid', 'dysfunction', 'run'],
 ['thank',
  'lyft',
  'credit',
  'use',
  'cause',
  'offer',
  'wheelchair',
  'vans',
  'pdx',
  'disapointed',
  'getthanked'],
 ['bihday', 'majesty'],
 ['model', 'love', 'take', 'time', 'ur']]

In [50]:
# приводим тексты твитов в формат, который принимает векторайзер (список предложений)
all_text_lemmed = []
for row in texts:
    row_text = ""
    for word in row:
        row_text = row_text + " " + word
    all_text_lemmed.append(row_text.strip())

In [52]:
# что у нас получилось после преобразования
all_text_lemmed[:10]

['father dysfunctional selfish drag kid dysfunction run',
 'thank lyft credit use cause offer wheelchair vans pdx disapointed getthanked',
 'bihday majesty',
 'model love take time ur',
 'factsguide society motivation',
 'huge fan fare big talk leave chaos pay dispute get allshowandnogo',
 'camp tomorrow danny',
 'next school year year exams think school exams hate imagine actorslife revolutionschool girl',
 'love land allin cavs champion cleveland clevelandcavaliers',
 'welcome gr']

In [53]:
# Создаем the Bag-of-Words модель
bag_of_words = count_vectorizer.fit_transform(all_text_lemmed)

In [54]:
# Отобразим Bag-of-Words модель как DataFrame
feature_names2 = count_vectorizer.get_feature_names()
bow_df2 = pd.DataFrame(bag_of_words.toarray(), columns = feature_names2)

In [55]:
bow_df2.head(2)

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,...,yesterday,yo,yoga,york,young,youth,youtube,yr,yrs,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Задание 2
Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [56]:
# создаём объект TfidfVectorizer и устанавливаем его параметры
# Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
# Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
# Исключим стоп-слова с помощью stop_words='english'.
# Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), analyzer='word', 
                                   binary=False, max_df = 0.9, max_features = 1000,
                                  stop_words='english')

In [67]:
# Создаем the Bag-of-Words модели для стеммированных текстов и для лемматизированных
bag_of_words_lemmed = tfidf_vectorizer.fit_transform(all_text_lemmed)
feature_names_lem = tfidf_vectorizer.get_feature_names()
bag_of_words_stemmed = tfidf_vectorizer.fit_transform(all_text_stemmed)
feature_names_stem = tfidf_vectorizer.get_feature_names()

In [68]:
# Отобразим Bag-of-Words модель как DataFrame
tfidf_df = pd.DataFrame(bag_of_words_lemmed.toarray(), columns = feature_names_lem)
tfidf_df2 = pd.DataFrame(bag_of_words_stemmed.toarray(), columns = feature_names_stem)

In [75]:
tfidf_df.iloc[28:31]

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,...,yesterday,yo,yoga,york,young,youth,youtube,yr,yrs,yummy
28,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29,0.0,0.0,0.0,0.0,0.0,0.345336,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
tfidf_df2.iloc[28:31]

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
28,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29,0.0,0.0,0.0,0.0,0.0,0.334456,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Задание 3 Натренируем gensim.models.Word2Vec модель на наших данных.

* Тренировать будем на токенизированных твитах combine_df['tweet_token']
* Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.
* Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.

In [80]:
#!pip install gensim

In [52]:
import gensim.downloader as api
from gensim.models import Word2Vec

In [150]:
texts = df['tweet_token'].to_list()

In [151]:
texts[:5]

[['when',
  'father',
  'is',
  'dysfunctional',
  'and',
  'is',
  'so',
  'selfish',
  'he',
  'drags',
  'his',
  'kids',
  'into',
  'his',
  'dysfunction',
  'run'],
 ['thanks',
  'for',
  'lyft',
  'credit',
  'can',
  'not',
  'use',
  'cause',
  'they',
  'do',
  'not',
  'offer',
  'wheelchair',
  'vans',
  'in',
  'pdx',
  'disapointed',
  'getthanked'],
 ['bihday', 'your', 'majesty'],
 ['model',
  'love',
  'you',
  'take',
  'with',
  'you',
  'all',
  'the',
  'time',
  'in',
  'ur'],
 ['factsguide', 'society', 'now', 'motivation']]

In [152]:
model = Word2Vec(sentences=texts, size=200, window=5, min_count=2, 
                 sg = 1, hs = 0, negative = 10, workers= 32, seed = 34)

In [153]:
len(texts)

49159

In [154]:
model.train(texts, total_examples=len(texts), epochs=20)

(9128456, 11676740)

## Задание 4 
Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".

In [155]:
model.wv.most_similar(positive=["trump"])

[('donald', 0.5501392483711243),
 ('crony', 0.5327620506286621),
 ('unfit', 0.5262327194213867),
 ('suppoer', 0.5197625756263733),
 ('donaldtrump', 0.5164532661437988),
 ('dumptrump', 0.5098274350166321),
 ('impeachment', 0.500749945640564),
 ('conman', 0.5001658201217651),
 ('unfavorability', 0.49976640939712524),
 ('phony', 0.49667587876319885)]

In [156]:
model.wv.most_similar(positive=["dinner"])

[('bihdaydinner', 0.5735843181610107),
 ('bolognese', 0.5555020570755005),
 ('cookout', 0.5464305877685547),
 ('shawarma', 0.5434701442718506),
 ('sissy', 0.5316126346588135),
 ('waterloo', 0.5298169851303101),
 ('spaghetti', 0.527080237865448),
 ('burritos', 0.5263620615005493),
 ('tacotuesday', 0.5226019620895386),
 ('lastnight', 0.5163992047309875)]

In [159]:
model.wv.most_similar(positive=["disapointed"])

KeyError: "word 'disapointed' not in vocabulary"

## Задание 5
Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).
Давайте проверим векторное представление любого слова из нашего корпуса, например "food".

In [29]:
my_word_vector = model.wv['food']
my_word_vector

array([ 1.0487982e-01,  7.1753567e-01,  1.6437873e-01, -4.9533224e-04,
       -4.0243381e-01,  2.4130361e-01, -1.7375448e-01,  1.9668108e-01,
        5.1833510e-01,  7.2512947e-02, -3.7711841e-01, -1.3450913e-01,
       -6.1501622e-01,  4.2265847e-02,  1.0030919e+00, -2.6114342e-01,
        2.4443835e-02,  5.6554186e-01,  3.3014292e-01,  3.7453169e-01,
        2.4988325e-01, -8.7458968e-02,  6.3764280e-01, -4.2551327e-01,
        7.8718758e-01, -3.1817943e-01,  1.2278194e-01, -1.0029565e+00,
        3.8749295e-01, -2.3035859e-01, -4.2597029e-01, -3.7988597e-01,
        4.9015749e-01, -6.0321999e-01, -3.7664428e-01,  3.8577858e-01,
       -1.6822539e-01,  5.6258911e-01, -4.2373922e-01, -1.2999626e-01,
       -2.8590664e-01,  5.5313420e-01, -2.2693746e-01, -1.3608521e-01,
       -6.4299113e-01,  2.0960350e-01, -1.3274111e-01, -4.9843445e-01,
       -1.3675939e-01,  2.6674110e-01,  4.2237278e-02,  4.3394662e-02,
       -5.5560046e-01, -1.2632187e-01, -4.4564524e-01, -4.6242688e-02,
      

In [34]:
len(my_word_vector), type(my_word_vector), my_word_vector.shape

(200, numpy.ndarray, (200,))

In [38]:
my_word_vector.reshape((1, 200))

array([[ 1.0487982e-01,  7.1753567e-01,  1.6437873e-01, -4.9533224e-04,
        -4.0243381e-01,  2.4130361e-01, -1.7375448e-01,  1.9668108e-01,
         5.1833510e-01,  7.2512947e-02, -3.7711841e-01, -1.3450913e-01,
        -6.1501622e-01,  4.2265847e-02,  1.0030919e+00, -2.6114342e-01,
         2.4443835e-02,  5.6554186e-01,  3.3014292e-01,  3.7453169e-01,
         2.4988325e-01, -8.7458968e-02,  6.3764280e-01, -4.2551327e-01,
         7.8718758e-01, -3.1817943e-01,  1.2278194e-01, -1.0029565e+00,
         3.8749295e-01, -2.3035859e-01, -4.2597029e-01, -3.7988597e-01,
         4.9015749e-01, -6.0321999e-01, -3.7664428e-01,  3.8577858e-01,
        -1.6822539e-01,  5.6258911e-01, -4.2373922e-01, -1.2999626e-01,
        -2.8590664e-01,  5.5313420e-01, -2.2693746e-01, -1.3608521e-01,
        -6.4299113e-01,  2.0960350e-01, -1.3274111e-01, -4.9843445e-01,
        -1.3675939e-01,  2.6674110e-01,  4.2237278e-02,  4.3394662e-02,
        -5.5560046e-01, -1.2632187e-01, -4.4564524e-01, -4.62426

## Задание 6
Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.

Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size))
и поделить финальный вектор на количество слов в твите.
На выходе должен получиться wordvec_df.shape = (49159, 200).


In [22]:
df.head(3)

,id,label,tweet,tweet_token,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,"[thanks, for, lyft, credit, can, not, use, cau...","[thank, lyft, credit, use, caus, offer, wheelc...","[thank, lyft, credit, use, cause, offer, wheel..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesti]","[bihday, majesty]"


In [ ]:
vector = np.zeros(200)
    words_counter = 0
    
    for word in tweet:
        try:
            vector += modelW2V.wv[word]
            words_counter += 1
        except:
            pass
    return (vector / words_counter).tolist()

In [196]:
size = 200
tweet_averages = []
for idx, row in enumerate(df['tweet_token'][:]):
    average_per_tweet = None
    vec = np.zeros(200)
    counter = 0
    for ind, word in enumerate(row):
        #print(idx, word)
        try:
            word_vector = model.wv[word]#.reshape((1, 200)) 
            vec = vec + word_vector
            counter = counter +1
            #print(idx, word, word_vector)
        except:
            pass
    average_per_tweet = (vec/counter)
    tweet_averages.append(average_per_tweet)
    #print(average_per_tweet)
            
#         vec += model.wv[word].reshape((1, size)) 
#     average_per_tweet = vec / (ind+1)
#     print(average_per_tweet)

/Users/artem/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


In [197]:
wordvec_df = pd.DataFrame(tweet_averages)
wordvec_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.157515,0.206160,0.240572,0.095627,0.342026,-0.164991,-0.195331,0.147425,0.130821,0.044960,...,-0.007398,0.013640,-0.110190,0.177521,0.106337,-0.387834,0.149893,0.411490,-0.086332,-0.267600
1,0.099679,0.222602,0.114998,0.011011,0.188402,-0.048495,-0.142211,0.206281,0.384407,-0.152005,...,-0.124551,0.019045,-0.099375,0.190515,0.068947,-0.198821,0.341550,0.362512,0.152765,-0.090385
2,0.317678,-0.127611,-0.241864,-0.143185,0.222966,-0.178260,-0.156288,0.284328,0.674881,-0.172447,...,0.109332,0.322368,-0.172000,0.188761,-0.075118,-0.536051,0.488682,0.605482,0.223515,-0.074148
3,0.074275,-0.001674,-0.168419,0.101925,0.379402,-0.157882,0.003343,0.165702,0.336220,0.031203,...,-0.130294,-0.164786,-0.157801,0.053534,0.211267,-0.189855,0.141836,0.411199,-0.083954,0.137592
4,0.019279,0.059107,0.133410,0.055024,0.243223,-0.249457,0.039960,0.170736,0.080388,-0.263483,...,0.052415,-0.111762,-0.281139,0.215636,-0.068879,-0.310696,-0.189842,0.640223,0.089527,-0.304095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0.382059,0.380581,0.068596,-0.058928,0.104340,-0.233776,-0.315963,0.014744,0.138054,0.188783,...,0.187803,-0.106518,0.306282,-0.016223,0.282827,-0.453220,0.122283,0.521535,0.264606,-0.245570
49155,0.114690,0.279147,0.175439,0.014056,0.315212,-0.109102,-0.251520,-0.132618,0.175838,-0.055309,...,-0.165045,-0.095527,-0.067403,0.162235,-0.084300,-0.387289,0.282083,0.477061,0.120090,-0.222803
49156,0.418465,0.085668,0.015495,0.090045,0.146709,-0.118686,-0.022246,0.075554,0.164491,-0.151903,...,-0.186417,0.145597,-0.045228,0.090908,0.040118,-0.358991,0.020523,0.217143,0.104946,-0.163964
49157,0.058645,-0.120679,-0.185780,0.179037,0.041425,-0.222860,0.030920,-0.160227,0.062575,-0.027752,...,-0.126250,-0.284861,-0.148850,0.272023,-0.208541,-0.299925,0.591013,0.718859,0.226171,-0.334430
